In [3]:
import pygsheets
service_account = pygsheets.authorize(service_account_file='JSONs/spreadsheet-automator-436913-2e0d40dde567.json')
sheet = service_account.open_by_url('https://docs.google.com/spreadsheets/d/1DVMUAbhfNE6OF3_yydTQnpOPoBGvz6ESliRkIkhVsPU/edit?usp=sharing')

In [6]:
sheet = service_account.open_by_url('https://docs.google.com/spreadsheets/d/1DVMUAbhfNE6OF3_yydTQnpOPoBGvz6ESliRkIkhVsPU/edit?usp=sharing')
test = sheet.worksheet('title', 'Test')

In [ ]:
test.get_as_df()

In [9]:
test_df = test.get_as_df()

In [10]:
test_df['random_numbers'] = 1

In [21]:
from dotenv import load_dotenv
import pandas as pd
import os
import requests

load_dotenv(override=True)
api_key = os.environ.get('api_key')
gameName = 'otaldomirudo'
tagLine = 'apelo'

def get_puuid(summonerId=None, gameName=None, tagLine=None, region='br1'):
    '''Gets the puuid from a summonerId or riot_id and riot_tag
    
    Args:
        summonerId (str, optional): Summoner ID. Default = None
        gameName (str, optional): Riot ID. Default = None
        tagline (str, optional): Riot tag. Default = None
        region (str, optional): Region. Default = br1
    
    Returns:
        str: puuid
    ''' 
    
    root_url = f'https://{region}.api.riotgames.com/'

    if summonerId is not None:
        endpoint = 'lol/summoner/v4/summoners/'
        link = root_url + endpoint + summonerId + '?api_key=' + api_key
        print('o link está saindo:', link)
        response = requests.get(link)
    else:
        endpoint = f'riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}'
        link = root_url + endpoint + '?api_key=' + api_key
        response = requests.get(link)
        
    return response.json()['puuid']

def get_ladder(top=300):
    
    root_url = 'https://br1.api.riotgames.com/'
    chall = '/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5'
    gm = '/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5'
    masters = '/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5'

    masters_response = requests.get(root_url + masters + '?api_key=' + api_key)
    gm_response = requests.get(root_url + gm + '?api_key=' + api_key)
    chall_response = requests.get(root_url + chall + '?api_key=' + api_key)

    masters_df = pd.DataFrame(masters_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
    gm_df = pd.DataFrame(gm_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
    chall_df = pd.DataFrame(chall_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
    
    ladder = pd.concat([chall_df, gm_df, masters_df])[:top].reset_index(drop=True)
    ladder = ladder.drop(columns='rank').reset_index(drop=False).rename(columns={'index':'rank'})
    ladder['rank'] += 1

    return ladder

def get_match_history(puuid=None, region='americas', nMatches=100):
    root_url = f'https://{region}.api.riotgames.com/'
    endpoint = f'lol/match/v5/matches/by-puuid/{puuid}/ids'
    link = root_url + endpoint + '?start=0&count=' + str(nMatches) + '&api_key=' + api_key
    print(link)
    response = requests.get(link)
    return response.json()

def get_match_data_from_id(region='americas', matchId=None):
    root_url = f'https://{region}.api.riotgames.com/'
    endpoint = f'lol/match/v5/matches/{matchId}'
    link = root_url + endpoint + '?api_key=' + api_key
    print(link)
    response = requests.get(link)
    return response.json()

def process_match_json(match_json, puuid):


    metadata = match_json['metadata']
    info = match_json['info']
    players = info['participants']
    participants = metadata['participants']
    teams = info['teams']
    player = players[participants.index(puuid)]
    perks = player['perks']
    stats = perks['statPerks']
    styles = perks['styles']

    primary = styles[0]
    secondary = styles[1]

    side_dict = {
        100:'blue',
        200:'red'
    }

    match_id = metadata['matchId']

    game_creation = info['gameCreation']
    game_duration = info['gameDuration']
    game_start_timestamp = info['gameStartTimestamp']
    game_end_timestamp = info['gameEndTimestamp']
    patch = info['gameVersion']

    riot_id = player['riotIdGameName']
    riot_tag = player['riotIdTagline']
    summoner_id = player['summonerId']
    summoner_name = player['summonerName']

    side = side_dict[player['teamId']]
    win = player['win']

    champ_Id = player['championId']
    champ_transform = player['championTransform']
    champ_level = player['champLevel']
    team_position = player['teamPosition']

    gold_earned = player['goldEarned']
    neutral_minions_killed = player['neutralMinionsKilled']
    total_minions_killed = player['totalMinionsKilled']
    total_ally_jungle_minions_killed = player['totalAllyJungleMinionsKilled']
    total_enemy_jungle_minions_killed = player['totalEnemyJungleMinionsKilled']

    kills = player['kills']
    deaths = player['deaths']
    assists = player['assists']
    first_blood = player['firstBloodKill']

    total_damage_dealt = player['totalDamageDealtToChampions']
    total_damage_shielded = player['totalDamageShieldedOnTeammates']
    total_damage_taken = player['totalDamageTaken']
    total_damage_healed = player['totalHealsOnTeammates']
    total_time_cc_dealt = player['totalTimeCCDealt']

    early_surrender = player['gameEndedInEarlySurrender']
    surrender = player['gameEndedInSurrender']


    item0 = player['item0']
    item1 = player['item1']
    item2 = player['item2']
    item3 = player['item3']
    item4 = player['item4']
    item5 = player['item5']
    item6 = player['item6']



    summoner_1_id = player['summoner1Id']
    summoner_2_id = player['summoner2Id']

    wards_placed = player['wardsPlaced']
    wards_killed = player['wardsKilled']
    vision_score = player['visionScore']
    detector_wards_placed = player['detectorWardsPlaced']

    defense = stats['defense']
    flex = stats['flex']
    offense = stats['offense']

    primary_style = primary['style']
    secondary_style = secondary['style']

    primary_keystone = primary['selections'][0]['perk']
    primary_perk_1 = primary['selections'][1]['perk']
    primary_perk_2 = primary['selections'][2]['perk']
    primary_perk_3 = primary['selections'][3]['perk']

    secondary_perk_1 = secondary['selections'][0]['perk']
    secondary_perk_2 = secondary['selections'][1]['perk']

    objectives_stolen = player['objectivesStolen']
    objectives_stolen_assists = player['objectivesStolenAssists']


    matchDF = pd.DataFrame({
        'match_id': [match_id],
        'participants': [participants],
        'game_creation':[game_creation],
        'game_duration':[game_duration],
        'game_start_timestamp':[game_start_timestamp],
        'game_end_timestamp':[game_end_timestamp],
        'patch':[patch],
        'puuid':[puuid],
        'riot_id':[riot_id],
        'riot_tag':[riot_tag],
        'summoner_id':[summoner_id],
        'summoner_name':[summoner_name],
        'side':[side],
        'win':[win],
        'team_position':[team_position],
        'champion':[champ_Id],
        'champion_transform':[champ_transform],
        'champion_level':[champ_level],
        'kills':[kills],
        'deaths':[deaths],
        'assists':[assists],
        'summoner1_id':[summoner_1_id],
        'summoner2_id':[summoner_2_id],
        'gold_earned':[gold_earned],
        'total_minions_killed':[total_minions_killed],
        'neutral_minions_killed':[neutral_minions_killed],
        'total_ally_jungle_minions_killed':[total_ally_jungle_minions_killed],
        'total_enemy_jungle_minions_killed':[total_enemy_jungle_minions_killed],
        'early_surrender':[early_surrender],
        'surrender':[surrender],
        'first_blood':[first_blood],
        'objectives_stolen':[objectives_stolen],
        'objectives_stolen_assists':[objectives_stolen_assists],
        'total_damage_dealt':[total_damage_dealt],
        'total_damage_taken':[total_damage_taken],
        'total_damage_shielded':[total_damage_shielded],
        'total_damage_healed':[total_damage_healed],
        'total_time_crowd_controled':[total_time_cc_dealt],
        'vision_score':[vision_score],
        'wards_killed':[wards_killed],
        'wards_placed':[wards_placed],
        'control_wards_placed':[detector_wards_placed],
        'item0':[item0],
        'item1':[item1],
        'item2':[item2],
        'item3':[item3],
        'item4':[item4],
        'item5':[item5],
        'item6':[item6],
        'perk_keystone':[primary_keystone],
        'perk_primary_row_1':[primary_perk_1],
        'perk_primary_row_2':[primary_perk_2],
        'perk_primary_row_3':[primary_perk_3],
        'perk_secondary_row_1':[secondary_perk_1],
        'perk_secondary_row_2':[secondary_perk_2],
        'perk_primary_style':[primary_style],
        'perk_secondary_style':[secondary_style],
        'perk_shard_defense':[defense],
        'perk_shard_flex':[flex],
        'perk_shard_offense':[offense],
    })

    return matchDF

def attach_matches():
    match_ids = get_match_history('e4MYUet8Ezaa9xvsSImfPSyPKIWzoGLqTHy9POmy_fQslDTYZB3Hf7NE3BmkWEwioF76RJ7TWXJFww',nMatches=20,)

    def json_extract(obj, key):

        arr = []

        def extract(obj, arr, key):
            if isinstance(obj, dict):
                for k, v in obj.items():
                    if k== key:
                        arr.append(v)
                    elif isinstance(v, (dict, list)):
                        extract(v, arr, key)
            elif isinstance(obj, list):
                for item in obj:
                    extract(item, arr, key)
            
            return arr
        
        values = extract(obj, arr, key)
        return values

    df = pd.DataFrame()
    for match_id in match_ids:
        game = get_match_data_from_id(matchId=match_id)
        matchDF = process_match_json(game, puuid='e4MYUet8Ezaa9xvsSImfPSyPKIWzoGLqTHy9POmy_fQslDTYZB3Hf7NE3BmkWEwioF76RJ7TWXJFww')
        df = pd.concat([df, matchDF])

    perk = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/perks.json'
    perk_styles = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/perkstyles.json'
    items = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/items.json'

    perk_json = requests.get(perk).json()
    perk_styles_json = requests.get(perk_styles).json()
    items_json = requests.get(items).json()

    perk_ids = json_extract(perk_json, 'id')
    perk_names = json_extract(perk_json, 'name')
    perk_dict = dict(map(lambda i, j : (int(i), j), perk_ids, perk_names)) 

    perk_styles_ids = json_extract(perk_styles_json, 'id')
    perk_styles_names = json_extract(perk_styles_json, 'name')
    perk_styles_dict = dict(map(lambda i, j : (int(i), j), perk_styles_ids, perk_styles_names))

    items_ids = json_extract(items_json, 'id')
    items_names = json_extract(items_json, 'name')
    items_dict = dict(map(lambda i, j : (int(i), j), items_ids, items_names)) 

    df = df.replace(perk_dict).replace(perk_styles_dict).replace(items_dict)
    
    return df

df = attach_matches()
df


https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/e4MYUet8Ezaa9xvsSImfPSyPKIWzoGLqTHy9POmy_fQslDTYZB3Hf7NE3BmkWEwioF76RJ7TWXJFww/ids?start=0&count=20&api_key=RGAPI-1694fb66-bf51-4fbf-8662-2a638beed8fd
https://americas.api.riotgames.com/lol/match/v5/matches/status?api_key=RGAPI-1694fb66-bf51-4fbf-8662-2a638beed8fd


KeyError: 'metadata'